This code opens pmaps from files and uses ipywidgets to do some basic interpretatoin

Neil Symington
neil.symington@ga.gov.au

In [1]:
%matplotlib widget

In [11]:
import geopandas as gpd
import numpy as np
import pandas as pd
import aseg_gdf2
from shapely.geometry import Point
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [12]:
def find_nearest_point_ind(gdf, pt):
    
    dist = gdf.distance(pt)
    # retrun the nearest index and the distance
    return dist.idxmin(), dist.min()


In [13]:
infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\run3\output\lines\rjmcmc_fixed"

# Read in a summary of the points

dat = aseg_gdf2.read(infile)

df_AEM = dat.df()

xs = df_AEM['easting '].values
ys = df_AEM['northing '].values

df_AEM['geometry'] = [Point(xs[i],ys[i]) for i in range(len(xs))]

gdf_AEM = gpd.GeoDataFrame(df_AEM)

In [14]:
# Open dataframe to extract the pmap file from the fiducial

infile = r'C:\Users\PCUser\Desktop\NSC_data\data\AEM\HE\garjmcmc\run3\output\fid_2_pmap.csv'

df = pd.read_csv(infile)

In [15]:
# Now we try adding a coordinate finding the nearest fiducial and plotting it

x, y = 736689, 8620360


ind, distance = find_nearest_point_ind(gdf_AEM, Point(x,y))

near_fid = gdf_AEM.iloc[ind]

In [16]:
# Now open the pmap file as a dictionary

infile = df[df['fiducial'] == near_fid['fiducial ']]['matfile'].values[0]

D = loadmat(infile)

In [41]:
print(D['M'].dtype)

[('binfile', 'O'), ('comment', 'O'), ('flightnumber', 'O'), ('linenumber', 'O'), ('fidnumber', 'O'), ('easting', 'O'), ('northing', 'O'), ('ndata', 'O'), ('starttime', 'O'), ('endtime', 'O'), ('samplingtime', 'O'), ('nchain', 'O'), ('nsample', 'O'), ('nburnin', 'O'), ('nnuisance', 'O'), ('nl_min', 'O'), ('nl_max', 'O'), ('p_param', 'O'), ('pmin', 'O'), ('pmax', 'O'), ('npcells', 'O'), ('v_param', 'O'), ('vmin', 'O'), ('vmax', 'O'), ('nvcells', 'O'), ('sd_v', 'O'), ('sd_p', 'O'), ('sd_bd_v', 'O'), ('ar_birth', 'O'), ('ar_death', 'O'), ('ar_value', 'O'), ('ar_move', 'O'), ('ar_nuisance', 'O'), ('lhist', 'O'), ('cp', 'O'), ('f', 'O'), ('conv', 'O')]


In [114]:
burnin

50000

In [116]:
fig, ax = plt.subplots()


for i in range(nchains):
    sample = D['M']['conv'][0,0]['sample'][0,i].flatten()
    misfits = D['M']['conv'][0,0]['misfit'][0,i].flatten()
    ax.plot(sample, misfits/ndata)

ax.plot([1, nsample], [1,1], 'k')
ax.plot([burnin, burnin],[0.1,1e4], 'k')
ax.set_xlim([1, nsample])
ax.set_ylim(0.1, 1e4)

ax.set_xscale('log')
ax.set_yscale('log')
plt.show()

C:\Users\PCUser\Anaconda3\envs\garjmcmctdem\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [113]:
# Get the pdf

freq = D['M']['f'][0,0]

pdf = freq / freq.sum(axis =1)[0]

pdf[pdf == 0] = np.nan

# Get the changepoint histogram

cp = D['M']['cp'][0,0].flatten()

cp_pdf = D['M']['cp'][0,0] / freq.sum(axis =1)[0]

# Get the extent

extent = [D['M']['vmin'][0,0][0,0], D['M']['vmax'][0,0][0,0], D['M']['pmax'][0,0][0,0], D['M']['pmin'][0,0][0,0]]

# Get the percentile 

p10 = near_fid[[x for x in gdf_AEM.columns if x.startswith('conductivity_p10')]].astype(np.float).values
p50 = near_fid[[x for x in gdf_AEM.columns if x.startswith('conductivity_p50')]].astype(np.float).values
p90 = near_fid[[x for x in gdf_AEM.columns if x.startswith('conductivity_p90')]].astype(np.float).values
mean = near_fid[[x for x in gdf_AEM.columns if x.startswith('conductivity_mean')]].astype(np.float).values
depth_cells = near_fid[[x for x in gdf_AEM.columns if x.startswith('depth')]].astype(np.float).values

laybins = D['M']['lhist'][0,0][0,0][1].flatten()
laycounts = D['M']['lhist'][0,0][0,0][2].flatten() / freq.sum(axis =1)[0]

nsample = D['M']['nsample'][0,0][0,0]

ndata = int(D['M']['ndata'][0,0][0,0])
nsamples = np.int(D['M']['nsample'][0,0][0,0])
nchains = np.int(D['M']['nchain'][0,0][0,0])
burnin = int(D['M']['nburnin'][0,0][0,0])


In [129]:
fig = plt.figure(figsize = (5,10))

fig.canvas.layout.width = '12in'
fig.canvas.layout.height= '8in'

ax1 = fig.add_axes([0.05, 0.2, 0.35, 0.7])
ax2 = fig.add_axes([0.45, 0.2, 0.2, 0.7])
ax3 = fig.add_axes([0.72, 0.2, 0.2, 0.35])
ax4 = fig.add_axes([0.72, 0.6, 0.2, 0.3])
cbar_ax = fig.add_axes([0.05, 0.1, 0.35, 0.02])


# Plot probability map
im = ax1.imshow(pdf, extent = extent, aspect = 'auto', cmap = 'rainbow')
#  PLot the median, and percentile plots
ax1.plot(np.log10(p10), depth_cells, c = 'k',linestyle='dashed', label = 'p10')
ax1.plot(np.log10(p90), depth_cells, c = 'k',linestyle='dashed', label = 'p90')
ax1.plot(np.log10(p50), depth_cells, c = 'k',label = 'p50')
ax1.plot(np.log10(mean), depth_cells, c = 'grey',label = 'mean')

ax1.set_title('probability map')
ax1.set_ylabel('depth (mBGL)')
ax1.set_xlabel('log10 conductivity (S/m)')
ax1.grid(which = 'both')
ax1.legend()

ax2.plot(cp_pdf, depth_cells)
ax2.set_ylim(ax2.get_ylim()[::-1])
ax2.set_title('change point probability')
ax2.set_ylim(D['M']['pmax'][0,0][0,0], D['M']['pmin'][0,0][0,0])

ax3.bar(laybins, height = laycounts)

ax3.set_xlabel ("no of layers")

for i in range(nchains):
    sample = D['M']['conv'][0,0]['sample'][0,i].flatten()
    misfits = D['M']['conv'][0,0]['misfit'][0,i].flatten()
    ax4.plot(sample, misfits/ndata)

ax4.plot([1, nsample], [1,1], 'k')
ax4.plot([burnin, burnin],[0.1,1e4], 'k')
ax4.set_xlim([1, nsample])
ax4.set_ylim(0.1, 1e4)

ax4.set_xscale('log')
ax4.set_yscale('log')

ax4.set_xlabel("sample #")
ax4.set_ylabel("Normalised misfit")

fig.colorbar(im, cax=cbar_ax, orientation='horizontal')


C:\Users\PCUser\Anaconda3\envs\garjmcmctdem\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …